# ETL Processes
Use this notebook to develop the ETL process for each of your tables before completing the `etl.py` file to load the whole datasets.

In [1]:
import os
import glob
import psycopg2
import pandas as pd

import sql_queries

In [2]:
conn = psycopg2.connect("host=127.0.0.1 dbname=sparkifydb user=student password=student")
cur = conn.cursor()

In [3]:
def get_files(filepath):
    all_files = []
    for root, dirs, files in os.walk(filepath):
        files = glob.glob(os.path.join(root,'*.json'))
        for f in files :
            all_files.append(os.path.abspath(f))
    
    return all_files

# Process `song_data`
In this first part, you'll perform ETL on the first dataset, `song_data`, to create the `songs` and `artists` dimensional tables.

Let's perform ETL on a single song file and load a single record into each table to start.
- Use the `get_files` function provided above to get a list of all song JSON files in `data/song_data`
- Select the first song in this list
- Read the song file and view the data

In [4]:
song_filepath = 'data/song_data'
song_files = get_files(song_filepath)

In [5]:
song_files

['/home/workspace/data/song_data/A/A/A/TRAAAFD128F92F423A.json',
 '/home/workspace/data/song_data/A/A/A/TRAAARJ128F9320760.json',
 '/home/workspace/data/song_data/A/A/A/TRAAAEF128F4273421.json',
 '/home/workspace/data/song_data/A/A/A/TRAAABD128F429CF47.json',
 '/home/workspace/data/song_data/A/A/A/TRAAAVO128F93133D4.json',
 '/home/workspace/data/song_data/A/A/A/TRAAAMO128F1481E7F.json',
 '/home/workspace/data/song_data/A/A/A/TRAAAVG12903CFA543.json',
 '/home/workspace/data/song_data/A/A/A/TRAAAMQ128F1460CD3.json',
 '/home/workspace/data/song_data/A/A/A/TRAAADZ128F9348C2E.json',
 '/home/workspace/data/song_data/A/A/A/TRAAAAW128F429D538.json',
 '/home/workspace/data/song_data/A/A/A/TRAAAPK128E0786D96.json',
 '/home/workspace/data/song_data/A/A/C/TRAACLV128F427E123.json',
 '/home/workspace/data/song_data/A/A/C/TRAACNS128F14A2DF5.json',
 '/home/workspace/data/song_data/A/A/C/TRAACTB12903CAAF15.json',
 '/home/workspace/data/song_data/A/A/C/TRAACZK128F4243829.json',
 '/home/workspace/data/so

In [6]:
df_song = pd.read_json(song_files[0], lines=True)
df_song.head()

,artist_id,artist_latitude,artist_location,artist_longitude,artist_name,duration,num_songs,song_id,title,year
0,ARXR32B1187FB57099,NaN,,NaN,Gob,209.60608,1,SOFSOCN12A8C143F5D,Face the Ashes,2007


## #1: `songs` Table
#### Extract Data for Songs Table
- Select columns for song ID, title, artist ID, year, and duration
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `song_data`

In [7]:
song_data = df_song[['song_id', 'title', 'artist_id', 'year', 'duration']].values[0]
song_data

array(['SOFSOCN12A8C143F5D', 'Face the Ashes', 'ARXR32B1187FB57099', 2007,
       209.60608], dtype=object)

In [8]:
song_data = list(song_data)
song_data

['SOFSOCN12A8C143F5D', 'Face the Ashes', 'ARXR32B1187FB57099', 2007, 209.60608]

#### Insert Record into Song Table
Implement the `song_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song into the `songs` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songs` table in the sparkify database.

In [9]:
cur.execute(sql_queries.song_table_insert, song_data)
conn.commit()

Run `test.ipynb` to see if you've successfully added a record to this table.

## #2: `artists` Table
#### Extract Data for Artists Table
- Select columns for artist ID, name, location, latitude, and longitude
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `artist_data`

In [10]:
artist_data = df_song[['artist_id', 'artist_name', 'artist_location', 'artist_latitude', 'artist_longitude']].values[0]
artist_data

array(['ARXR32B1187FB57099', 'Gob', '', nan, nan], dtype=object)

In [11]:
artist_data = list(artist_data)
artist_data

['ARXR32B1187FB57099', 'Gob', '', nan, nan]

#### Insert Record into Artist Table
Implement the `artist_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song's artist into the `artists` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `artists` table in the sparkify database.

In [12]:
cur.execute(sql_queries.artist_table_insert, artist_data)
conn.commit()

Run `test.ipynb` to see if you've successfully added a record to this table.

# Process `log_data`
In this part, you'll perform ETL on the second dataset, `log_data`, to create the `time` and `users` dimensional tables, as well as the `songplays` fact table.

Let's perform ETL on a single log file and load a single record into each table.
- Use the `get_files` function provided above to get a list of all log JSON files in `data/log_data`
- Select the first log file in this list
- Read the log file and view the data

In [13]:
log_filepath = 'data/log_data'
log_files = get_files(log_filepath)

In [14]:
log_files

['/home/workspace/data/log_data/2018/11/2018-11-13-events.json',
 '/home/workspace/data/log_data/2018/11/2018-11-10-events.json',
 '/home/workspace/data/log_data/2018/11/2018-11-02-events.json',
 '/home/workspace/data/log_data/2018/11/2018-11-12-events.json',
 '/home/workspace/data/log_data/2018/11/2018-11-14-events.json',
 '/home/workspace/data/log_data/2018/11/2018-11-29-events.json',
 '/home/workspace/data/log_data/2018/11/2018-11-09-events.json',
 '/home/workspace/data/log_data/2018/11/2018-11-06-events.json',
 '/home/workspace/data/log_data/2018/11/2018-11-21-events.json',
 '/home/workspace/data/log_data/2018/11/2018-11-20-events.json',
 '/home/workspace/data/log_data/2018/11/2018-11-19-events.json',
 '/home/workspace/data/log_data/2018/11/2018-11-11-events.json',
 '/home/workspace/data/log_data/2018/11/2018-11-15-events.json',
 '/home/workspace/data/log_data/2018/11/2018-11-01-events.json',
 '/home/workspace/data/log_data/2018/11/2018-11-16-events.json',
 '/home/workspace/data/lo

In [15]:
df_log = pd.read_json(log_files[0], lines=True)
df_log.head()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,None,Logged In,Kevin,M,0,Arellano,NaN,free,"Harrisburg-Carlisle, PA",GET,Home,1.540007e+12,514,None,200,1542069417796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",66
1,Fu,Logged In,Kevin,M,1,Arellano,280.05832,free,"Harrisburg-Carlisle, PA",PUT,NextSong,1.540007e+12,514,Ja I Ty,200,1542069637796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",66
2,None,Logged In,Maia,F,0,Burke,NaN,free,"Houston-The Woodlands-Sugar Land, TX",GET,Home,1.540677e+12,510,None,200,1542071524796,"""Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebK...",51
3,All Time Low,Logged In,Maia,F,1,Burke,177.84118,free,"Houston-The Woodlands-Sugar Land, TX",PUT,NextSong,1.540677e+12,510,A Party Song (The Walk of Shame),200,1542071549796,"""Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebK...",51
4,Nik & Jay,Logged In,Wyatt,M,0,Scott,196.51873,free,"Eureka-Arcata-Fortuna, CA",PUT,NextSong,1.540872e+12,379,Pop-Pop!,200,1542079142796,Mozilla/5.0 (Windows NT 6.1; WOW64; Trident/7....,9


## #3: `time` Table
#### Extract Data for Time Table
- Filter records by `NextSong` action
- Convert the `ts` timestamp column to datetime
  - Hint: the current timestamp is in milliseconds
- Extract the timestamp, hour, day, week of year, month, year, and weekday from the `ts` column and set `time_data` to a list containing these values in order
  - Hint: use pandas' [`dt` attribute](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.dt.html) to access easily datetimelike properties.
- Specify labels for these columns and set to `column_labels`
- Create a dataframe, `time_df,` containing the time data for this file by combining `column_labels` and `time_data` into a dictionary and converting this into a dataframe

In [16]:
df_log = df_log[df_log.page == 'NextSong'] 
df_log.head()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
1,Fu,Logged In,Kevin,M,1,Arellano,280.05832,free,"Harrisburg-Carlisle, PA",PUT,NextSong,1.540007e+12,514,Ja I Ty,200,1542069637796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",66
3,All Time Low,Logged In,Maia,F,1,Burke,177.84118,free,"Houston-The Woodlands-Sugar Land, TX",PUT,NextSong,1.540677e+12,510,A Party Song (The Walk of Shame),200,1542071549796,"""Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebK...",51
4,Nik & Jay,Logged In,Wyatt,M,0,Scott,196.51873,free,"Eureka-Arcata-Fortuna, CA",PUT,NextSong,1.540872e+12,379,Pop-Pop!,200,1542079142796,Mozilla/5.0 (Windows NT 6.1; WOW64; Trident/7....,9
5,Quad City DJ's,Logged In,Chloe,F,0,Cuevas,451.44771,free,"San Francisco-Oakland-Hayward, CA",PUT,NextSong,1.540941e+12,506,C'mon N' Ride It (The Train) (LP Version),200,1542081112796,Mozilla/5.0 (Windows NT 5.1; rv:31.0) Gecko/20...,49
8,Hoobastank,Logged In,Noah,M,1,Chavez,232.17587,free,"Ogden-Clearfield, UT",PUT,NextSong,1.540971e+12,492,Born To Lead,200,1542085206796,Mozilla/5.0 (Windows NT 6.1; WOW64; rv:32.0) G...,94


In [17]:
df_log['ts'] = pd.to_datetime(df_log['ts'], unit='ms')  
df_log.head()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
1,Fu,Logged In,Kevin,M,1,Arellano,280.05832,free,"Harrisburg-Carlisle, PA",PUT,NextSong,1.540007e+12,514,Ja I Ty,200,2018-11-13 00:40:37.796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",66
3,All Time Low,Logged In,Maia,F,1,Burke,177.84118,free,"Houston-The Woodlands-Sugar Land, TX",PUT,NextSong,1.540677e+12,510,A Party Song (The Walk of Shame),200,2018-11-13 01:12:29.796,"""Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebK...",51
4,Nik & Jay,Logged In,Wyatt,M,0,Scott,196.51873,free,"Eureka-Arcata-Fortuna, CA",PUT,NextSong,1.540872e+12,379,Pop-Pop!,200,2018-11-13 03:19:02.796,Mozilla/5.0 (Windows NT 6.1; WOW64; Trident/7....,9
5,Quad City DJ's,Logged In,Chloe,F,0,Cuevas,451.44771,free,"San Francisco-Oakland-Hayward, CA",PUT,NextSong,1.540941e+12,506,C'mon N' Ride It (The Train) (LP Version),200,2018-11-13 03:51:52.796,Mozilla/5.0 (Windows NT 5.1; rv:31.0) Gecko/20...,49
8,Hoobastank,Logged In,Noah,M,1,Chavez,232.17587,free,"Ogden-Clearfield, UT",PUT,NextSong,1.540971e+12,492,Born To Lead,200,2018-11-13 05:00:06.796,Mozilla/5.0 (Windows NT 6.1; WOW64; rv:32.0) G...,94


In [18]:
time_data = [df_log.ts, df_log.ts.dt.hour, df_log.ts.dt.day, df_log.ts.dt.week, df_log.ts.dt.month, df_log.ts.dt.year, df_log.ts.dt.weekday]

In [19]:
column_labels = ['start_time', 'hour', 'day', 'week', 'month', 'year', 'weekday']

In [20]:
time_df = pd.DataFrame.from_dict(dict(zip(column_labels, time_data)))

In [21]:
time_df.head()

,start_time,hour,day,week,month,year,weekday
1,2018-11-13 00:40:37.796,0,13,46,11,2018,1
3,2018-11-13 01:12:29.796,1,13,46,11,2018,1
4,2018-11-13 03:19:02.796,3,13,46,11,2018,1
5,2018-11-13 03:51:52.796,3,13,46,11,2018,1
8,2018-11-13 05:00:06.796,5,13,46,11,2018,1


In [22]:
time_df.dtypes

start_time    datetime64[ns]
hour                   int64
day                    int64
week                   int64
month                  int64
year                   int64
weekday                int64
dtype: object

#### Insert Records into Time Table
Implement the `time_table_insert` query in `sql_queries.py` and run the cell below to insert records for the timestamps in this log file into the `time` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `time` table in the sparkify database.

In [23]:
for i, row in time_df.iterrows():
    cur.execute(sql_queries.time_table_insert, list(row))
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #4: `users` Table
#### Extract Data for Users Table
- Select columns for user ID, first name, last name, gender and level and set to `user_df`

In [24]:
user_df = df_log[['userId', 'firstName', 'lastName', 'gender', 'level']]

In [25]:
user_df.head()

,userId,firstName,lastName,gender,level
1,66,Kevin,Arellano,M,free
3,51,Maia,Burke,F,free
4,9,Wyatt,Scott,M,free
5,49,Chloe,Cuevas,F,free
8,94,Noah,Chavez,M,free


#### Insert Records into Users Table
Implement the `user_table_insert` query in `sql_queries.py` and run the cell below to insert records for the users in this log file into the `users` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `users` table in the sparkify database.

In [26]:
for i, row in user_df.iterrows():
    cur.execute(sql_queries.user_table_insert, row)
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #5: `songplays` Table
#### Extract Data and Songplays Table
This one is a little more complicated since information from the songs table, artists table, and original log file are all needed for the `songplays` table. Since the log file does not specify an ID for either the song or the artist, you'll need to get the song ID and artist ID by querying the songs and artists tables to find matches based on song title, artist name, and song duration time.
- Implement the `song_select` query in `sql_queries.py` to find the song ID and artist ID based on the title, artist name, and duration of a song.
- Select the timestamp, user ID, level, song ID, artist ID, session ID, location, and user agent and set to `songplay_data`

#### Insert Records into Songplays Table
- Implement the `songplay_table_insert` query and run the cell below to insert records for the songplay actions in this log file into the `songplays` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songplays` table in the sparkify database.

In [27]:
df_log[:2]

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
1,Fu,Logged In,Kevin,M,1,Arellano,280.05832,free,"Harrisburg-Carlisle, PA",PUT,NextSong,1.540007e+12,514,Ja I Ty,200,2018-11-13 00:40:37.796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",66
3,All Time Low,Logged In,Maia,F,1,Burke,177.84118,free,"Houston-The Woodlands-Sugar Land, TX",PUT,NextSong,1.540677e+12,510,A Party Song (The Walk of Shame),200,2018-11-13 01:12:29.796,"""Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebK...",51


In [28]:
for index, row in df_log.iterrows():

    # get songid and artistid from song and artist tables
    cur.execute(sql_queries.song_select, (row.song, row.artist, row.length))
    results = cur.fetchone()
    
    if results:
        songid, artistid = results
    else:
        songid, artistid = None, None

    # insert songplay record
    songplay_data = (index, row['ts'], row['userId'], row['level'], songid, artistid, row['sessionId'], row['location'], row['userAgent'])
    cur.execute(sql_queries.songplay_table_insert, songplay_data)
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

# Close Connection to Sparkify Database

In [29]:
conn.close()

# Implement `etl.py`
Use what you've completed in this notebook to implement `etl.py`.